In [2]:
pip install mediapipe pyautogui pandas pygetwindow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import math
import pyautogui
import time
import pygetwindow as gw
import socket
import numpy as np

soc = socket.socket()
hostname = "localhost"
port = 65436
soc.bind((hostname, port))
soc.listen(1)
print("Server is listening... Waiting for a client to connect.")
conn, addr = soc.accept()
print(f"Connected to {addr}")

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

debounce_time = 0.5  # Time in seconds
last_trigger_time = time.time()


def send_command_to_client(command):
    global conn
    if conn:
        try:
            conn.sendall(command.encode('utf-8'))
        except (ConnectionAbortedError, ConnectionResetError, BrokenPipeError) as e:
            conn = None

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def distance_close(results, frame, distance_threshold):
    if len(results.multi_hand_landmarks) == 2:
        hand_1_wrist = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.WRIST]
        hand_2_wrist = results.multi_hand_landmarks[1].landmark[mp_hands.HandLandmark.WRIST]

        wrist_1_x = int(hand_1_wrist.x * frame.shape[1])
        wrist_1_y = int(hand_1_wrist.y * frame.shape[0])
        wrist_2_x = int(hand_2_wrist.x * frame.shape[1])
        wrist_2_y = int(hand_2_wrist.y * frame.shape[0])

        distance = np.sqrt((wrist_2_x - wrist_1_x) ** 2 + (wrist_2_y - wrist_1_y) ** 2)
        return distance
    return None

def maximize_window():
    global last_trigger_time
    current_time = time.time()
    if current_time - last_trigger_time > debounce_time:  # Debounce check
        active_window = gw.getActiveWindow()
        if active_window:
            active_window.activate()
            time.sleep(0.5)
            active_window.maximize()
            send_command_to_client("maximize\n")
            last_trigger_time = current_time  # Update last trigger time

def restore_window():
    global last_trigger_time
    current_time = time.time()
    if current_time - last_trigger_time > debounce_time:  # Debounce check
        active_window = gw.getActiveWindow()
        if active_window:
            active_window.restore()
            screen_width, screen_height = pyautogui.size()
            half_width = screen_width // 2
            active_window.resizeTo(half_width, screen_height)
            active_window.moveTo(0, 0)
            send_command_to_client("restore\n")
            last_trigger_time = current_time  # Update last trigger time

def detect_gesture(hand_landmarks, previous_x, frame, swipe_threshold=60):
    global last_trigger_time
    x = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame.shape[1]

    if previous_x is not None:
        if x - previous_x > swipe_threshold:
            current_time = time.time()
            if current_time - last_trigger_time > debounce_time:  # Debounce check
                pyautogui.keyDown('alt')
                pyautogui.press('tab')
                pyautogui.keyUp('alt')

                msg = b"Swipe detected, Alt + Tab triggered!\n"
                try:
                    conn.send(msg)
                    print("Swipe detected, Alt + Tab triggered!")
                except Exception as e:
                    print(f"Error sending message: {e}")

                last_trigger_time = current_time  # Update last trigger time

    return x 

def volume_up():
    global last_trigger_time
    current_time = time.time()
    if current_time - last_trigger_time > debounce_time:  # Debounce check
        pyautogui.press('volumeup')
        print("Volume Up")
        send_command_to_client("volume_up\n")
        last_trigger_time = current_time  # Update last trigger time

def volume_down():
    global last_trigger_time
    current_time = time.time()
    if current_time - last_trigger_time > debounce_time:  # Debounce check
        pyautogui.press('volumedown')
        print("Volume Down")
        send_command_to_client("volume_down\n")
        last_trigger_time = current_time  # Update last trigger time

def is_open_hand(landmarks):
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    return (
        index_tip.y < landmarks[6].y and
        middle_tip.y < landmarks[10].y and
        ring_tip.y < landmarks[14].y and
        pinky_tip.y < landmarks[18].y
    )

def is_closed_fist(landmarks):
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    return (
        thumb_tip.x < landmarks[3].x and
        index_tip.y > landmarks[6].y and
        middle_tip.y > landmarks[10].y and
        ring_tip.y > landmarks[14].y and
        pinky_tip.y > landmarks[18].y
    )

def close_active_window():
    global last_trigger_time
    current_time = time.time()
    if current_time - last_trigger_time > debounce_time:  # Debounce check
        active_window = gw.getActiveWindow()
        if active_window:
            window_title = active_window.title
            print(f"Attempting to close: {window_title}")
            pyautogui.hotkey('ctrl', 'w')
            message = f"Closed fist detected! Application closed: {window_title}"
            send_command_to_client(message)
            print(f"Sent message: {message}")
        else:
            print("No active window detected.")
            send_command_to_client("No active window detected.")
        last_trigger_time = current_time  # Update last trigger time

movement_threshold = 60  # Adjust this based on screen size and sensitivity
previous_y = None
was_open_hand = False
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    previous_x = None
    while cap.isOpened():
        _, frame = cap.read()
        try:
            f_frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(f_frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)

            annotated_image = f_frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    thumb_x = thumb_tip.x
                    thumb_y = thumb_tip.y
                    index_x = index_tip.x
                    index_y = index_tip.y
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    previous_x = detect_gesture(hand_landmarks, previous_x, frame)

                    distance = calculate_distance(thumb_x, thumb_y, index_x, index_y)

                    wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y * frame.shape[0]

                
                    if previous_y is not None:
                        movement = wrist_y - previous_y
    
                        # Volume up if hand moves upward, down if downward
                        if movement < -movement_threshold:
                            volume_up()
                        elif movement > movement_threshold:
                            volume_down()
    
                    # Update previous_y to current y-coordinate of the wrist
                    previous_y = wrist_y

                    if distance > 0.2:
                        maximize_window()
                    elif distance < 0.1:
                        restore_window()

                distance = distance_close(results, frame, 450)
                if distance is not None:
                    cv2.putText(annotated_image, f"Distance: {int(distance)}", (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    if distance > 450:
                        print("Hands are far apart - Stopping code")
                        send_command_to_client("Hands are far apart - Stopping code\n")
                        send_command_to_client("quit\n")
                        break

                landmarks = hand_landmarks.landmark

                # Check gestures
                if is_open_hand(landmarks):
                    was_open_hand = True
                elif was_open_hand and is_closed_fist(landmarks):
                    print("Transition from open hand to closed fist detected! Closing active application...")
                    close_active_window()
                    send_command_to_client("Closed fist detected! Application closed.")
                    time.sleep(1)
                    was_open_hand = False


            
            else:
                previous_y = None

            cv2.imshow('MediaPipe Hands', annotated_image)

        except Exception as e: 
            print(f"Error: {e}")
            break

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
conn.close()
soc.close()


Server is listening... Waiting for a client to connect.
Connected to ('127.0.0.1', 58976)
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Transition from open hand to closed fist detected! Closing active application...
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Transition from open hand to closed fist detected! Closing active application...
Transition from open hand to closed fist detected! Closing active application...
Transition from open hand to closed fist detected! Closing active application...
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Volume Up
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Swipe detected, Alt + Tab triggered!
Hands are far apart -

In [ ]:

# code work avec 1 hand

import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

image_path = r"C:\Users\Fayroz\Pictures\Saved Pictures\marvel.jpg"
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

if image is None:
    print(f"Failed to load image from {image_path}. Check if the file exists and the path is correct.")
else:
    print("Image loaded successfully.")

image_height, image_width = 150, 200
image = cv2.resize(image, (image_width, image_height))

cap = cv2.VideoCapture(0)

x, y = 100, 100
previous_x, previous_y = None, None
freeze_movement = False  

def is_fist(hand_landmarks):
    """
    Detects if the hand is in a fist position.
    """
    finger_tips = [
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP,
    ]
    finger_bottoms = [
        mp_hands.HandLandmark.INDEX_FINGER_PIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
        mp_hands.HandLandmark.RING_FINGER_PIP,
        mp_hands.HandLandmark.PINKY_PIP,
    ]

    # Check if each finger tip is below or close to its bottom joint
    for tip, bottom in zip(finger_tips, finger_bottoms):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[bottom].y:
            return False
    return True

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    # Handle only one hand
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]  

        if is_fist(hand_landmarks):
            freeze_movement = True  # Freeze the movement
        else:
            freeze_movement = False  # Allow the movement to continue

        if not freeze_movement:
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            finger_x = int(index_tip.x * w)
            finger_y = int(index_tip.y * h)

            if previous_x is not None and previous_y is not None:
                dx = finger_x - previous_x
                dy = finger_y - previous_y

                x += dx
                y += dy

            previous_x, previous_y = finger_x, finger_y

        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    else:
        previous_x, previous_y = None, None

    x = max(0, min(x, frame.shape[1] - image_width))
    y = max(0, min(y, frame.shape[0] - image_height))

    y1, y2 = y, y + image_height
    x1, x2 = x, x + image_width
    frame[y1:y2, x1:x2] = image

    # Display the result
    cv2.imshow("Move Picture", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import pyautogui

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

screen_width, screen_height = pyautogui.size()

cap = cv2.VideoCapture(0)

previous_x, previous_y = None, None
freeze_movement = False  

def is_fist(hand_landmarks):
    """
    Detects if the hand is in a fist position.
    """
    finger_tips = [
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP,
    ]
    finger_bottoms = [
        mp_hands.HandLandmark.INDEX_FINGER_PIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
        mp_hands.HandLandmark.RING_FINGER_PIP,
        mp_hands.HandLandmark.PINKY_PIP,
    ]

    for tip, bottom in zip(finger_tips, finger_bottoms):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[bottom].y:
            return False
    return True

def is_finger_extended(hand_landmarks, finger_tip, finger_bottom):
    """
    Check if a finger is extended (tip is above bottom joint).
    """
    if hand_landmarks.landmark[finger_tip].y < hand_landmarks.landmark[finger_bottom].y:
        return True
    return False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]  # First hand is used for movement

        if is_fist(hand_landmarks):
            freeze_movement = True  # Freeze the movement
        else:
            freeze_movement = False  

        index_extended = is_finger_extended(hand_landmarks, mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.INDEX_FINGER_PIP)
        middle_extended = is_finger_extended(hand_landmarks, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP)

        if not freeze_movement:
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            h, w, _ = frame.shape
            finger_x = int(index_tip.x * w)
            finger_y = int(index_tip.y * h)

            if previous_x is not None and previous_y is not None:
                dx = finger_x - previous_x
                dy = finger_y - previous_y

                screen_x = int(finger_x * screen_width / w)
                screen_y = int(finger_y * screen_height / h)

                pyautogui.moveTo(screen_x, screen_y)

                if index_extended and middle_extended:
                    pyautogui.click()

            previous_x, previous_y = finger_x, finger_y

        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    else:
        previous_x, previous_y = None, None

    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
